In [ ]:
!pip install MLflow
import mlflow
mlflow.autolog()

In [4]:
# @title Importation données
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
ct = '/content/drive/MyDrive/QueryResults (1).csv'
df = pd.read_csv(ct)
df

Mounted at /content/drive


,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


In [5]:
# @title Filtre des tags

df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3]

<ipython-input-5-c9400e09b6b9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nb_tags'] = df['Tags'].apply(lambda x: len(x))


In [6]:
# @title Nettoyage du texte
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Nettoyage
def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']

def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


2023/11/24 16:06:42 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
# @title Encoding TF - IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténez les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

In [8]:
# @title Modélisation  LDA 2
from sklearn.decomposition import LatentDirichletAllocation

X = tfidf_df
# Création du modèle LDA avec 50 sujets
num_topics = 50
with mlflow.start_run(run_name='unsupervised') as run:
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)
    mlflow.sklearn.log_model(
        sk_model=lda_model,
        artifact_path="model",
        registered_model_name="unsupervised",
    )
# Obtenir les trois sujets les plus probables pour chaque ligne
top_n = 3
top_topics = lda_model.transform(X).argsort(axis=1)[:, -top_n:]

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [topics for topics in top_topics]

2023/11/24 16:08:09 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/11/24 16:08:09 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/11/24 16:08:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/11/24 16:08:14 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
Successfully registered model 'unsupervised'.
Created version '1' of model 'unsupervised'.


In [9]:
df[['Tags','lda_predict']]

,Tags,lda_predict
0,"[c#, java, asp.net]","[23, 48, 27]"
1,"[php, mysql, linux]","[45, 38, 34]"
2,"[swift, xcode, macos]","[0, 18, 28]"
3,"[javascript, php, html]","[49, 20, 48]"
4,"[c#, c, string]","[49, 18, 27]"
...,...,...
6639,"[c#, iphone, objective-c]","[32, 28, 27]"
6640,"[php, jquery, html]","[12, 45, 48]"
6641,"[javascript, jquery, css]","[27, 42, 11]"
6642,"[c#, asp.net, angularjs]","[24, 32, 10]"


In [10]:
# @title Récupération des sujets du LDA
res = df[['Tags','lda_predict']]
res

from collections import defaultdict

# Création d'un dictionnaire pour compter les correspondances
tag_counts = defaultdict(lambda: defaultdict(int))

# Parcours des lignes pour compter les correspondances entre les tags et les numéros
for index, row in df.iterrows():
    tags = row['Tags']
    lda_topics = row['lda_predict']

    # Comptage des correspondances entre les tags et les numéros
    for tag in tags:
        for topic in lda_topics:
            tag_counts[topic][tag] += 1


results = pd.DataFrame(tag_counts).T
results.fillna(0, inplace=True)
results = results.rename_axis('LDA Topic').reset_index()

word_to_topic = {}
cols = results.columns.drop('LDA Topic')

for word in cols:
    id_topic = results[word].idxmax()  # Trouver le numéro de sujet LDA avec le nombre le plus élevé d'occurrences
    word_to_topic[results.loc[id_topic, 'LDA Topic']] =  word
    results = results.drop(id_topic)

df['lda_predict'] = df['lda_predict'].apply(lambda topics: [word_to_topic.get(topic, 'Unknown') for topic in topics])

In [11]:
df[['Tags','lda_predict']]

,Tags,lda_predict
0,"[c#, java, asp.net]","[.net, asp.net, sql]"
1,"[php, mysql, linux]","[string, python-3.x, java]"
2,"[swift, xcode, macos]","[django, regex, ios]"
3,"[javascript, php, html]","[c#, pandas, asp.net]"
4,"[c#, c, string]","[c#, regex, sql]"
...,...,...
6639,"[c#, iphone, objective-c]","[mysql, ios, sql]"
6640,"[php, jquery, html]","[php, string, asp.net]"
6641,"[javascript, jquery, css]","[sql, wordpress, javascript]"
6642,"[c#, asp.net, angularjs]","[iphone, mysql, ruby-on-rails]"


In [12]:
# @title Scoring  LDA
gg_scores = []

for index, row in df.iterrows():
    real_tags = set(row['Tags'])
    lda_tags = set(row['lda_predict'])

    common_tags = real_tags.intersection(lda_tags)  # Tags communs entre les deux ensembles

    if len(common_tags) == 3:
        gg_scores.append(1)
    elif len(common_tags) == 2:
        gg_scores.append(2/3)
    elif len(common_tags) == 1:
        gg_scores.append(1/3)
    else:
        gg_scores.append(0)

# Calcul de la moyenne des scores "good guess"
gg = sum(gg_scores) / len(gg_scores)
print("gg = ", round(gg,2)*100," %")

gg =  20.0  %


In [ ]:
!pip install transformers

In [ ]:
# @title Extraction de caractéristiques avec BERT

from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np

# Modèle BERT et le tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Obtention du vecteur d'un document
def get_bert_embedding(text):
    # Tokenizer le texte et obtenir les embeddings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Prendre la sortie correspondant au [CLS] token
    embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return np.array(embeddings).flatten()


bert_features = np.array([get_bert_embedding(text) for text in df['Cleaned_Body']])

bert_df = pd.DataFrame(bert_features, columns=[f'bert_{i}' for i in range(bert_features.shape[1])])


In [ ]:
bert_df

In [ ]:
# @title Scoring algo supervisé avec BERT
bert_df = scaler.fit_transform(bert_df)
predicted_tags = modelisation(bert_df)
scoring(predicted_tags)